In [1]:
from sympy.diffgeom.rn import R2
from sympy.diffgeom import metric_to_Christoffel_2nd, TensorProduct, metric_to_Riemann_components,twoform_to_matrix,metric_to_Christoffel_1st
TP = TensorProduct

In [2]:
metric_to_Christoffel_2nd(TP(R2.dx, R2.dx) + TP(R2.dy, R2.dy))

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]

In [3]:
metric_to_Christoffel_2nd(R2.x*TP(R2.dx, R2.dx) + TP(R2.dy, R2.dy))

[[[1/(2*x), 0], [0, 0]], [[0, 0], [0, 0]]]

In [4]:
from sympy import symbols, pi, sqrt, atan2, cos, sin
from sympy.diffgeom import Manifold, Patch, CoordSystem
m = Manifold('M', 4)
p = Patch('P', m)
tt, xt, yt, zt = symbols('t x y z', real=True)
a, Q, rs = symbols('a Q rs')
#a=0
#Q=0

coordinates=CoordSystem("kerr",p,symbols=(tt, xt, yt, zt))

t, x, y, z=coordinates.base_scalars()

In [5]:
m=rs/2
r=x
delta=r**2-2*m*r+a**2+Q**2
rho2=r**2+(a**2)*(cos(y)**2)

In [6]:
dxi=coordinates.base_oneforms()

In [7]:
dx=dxi

In [8]:
dx

[dt, dx, dy, dz]

In [9]:
from sympy import Matrix
g =  Matrix([
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0,0]
])

In [10]:
g[0,0]=-a**2*sin(y)**2/rho2+delta/rho2
g[1,1]=-rho2/delta
g[2,2]=-rho2
g[3,3]=(delta*a**2*sin(y)**4-a**4*sin(y)**2-2*a**2*r**2*sin(y)**2-r**4*sin(y)**2)/rho2
g[0,3]=(a**3*sin(y)**2+a*r**2*sin(y)**2-delta*a*sin(y)**2)/rho2
g[3,0]=g[0,3]
g

Matrix([
[(-rs*x + x**2)/x**2,                    0,     0,               0],
[                  0, -x**2/(-rs*x + x**2),     0,               0],
[                  0,                    0, -x**2,               0],
[                  0,                    0,     0, -sin(y)**2*x**2]])

In [11]:
metric=0
for i in range(4):
    for j in range(4):
        metric+=g[i,j]*TP(dx[i],dx[j])
metric

(-rs*x + x**2)*TensorProduct(dt, dt)/x**2 - sin(y)**2*x**2*TensorProduct(dz, dz) - x**2*TensorProduct(dy, dy) - x**2*TensorProduct(dx, dx)/(-rs*x + x**2)

In [12]:
twoform_to_matrix(metric)

Matrix([
[-rs/x + 1,                    0,     0,               0],
[        0, -x**2/(-rs*x + x**2),     0,               0],
[        0,                    0, -x**2,               0],
[        0,                    0,     0, -sin(y)**2*x**2]])

In [13]:
ch=metric_to_Christoffel_2nd(metric)
ch

[[[0, rs/(2*(-rs + x)*x), 0, 0], [rs/(2*(-rs + x)*x), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[-rs*(rs - x)/(2*x**3), 0, 0, 0], [0, (rs - x)*(-(rs - 2*x)*x**2/(2*(-rs*x + x**2)**2) - x/(-rs*x + x**2))/x, 0, 0], [0, 0, rs - x, 0], [0, 0, 0, (rs - x)*sin(y)**2]], [[0, 0, 0, 0], [0, 0, 1/x, 0], [0, 1/x, 0, 0], [0, 0, 0, -sin(y)*cos(y)]], [[0, 0, 0, 0], [0, 0, 0, 1/x], [0, 0, 0, cos(y)/sin(y)], [0, 1/x, cos(y)/sin(y), 0]]]

In [14]:
dx_res=[0,0,0,0]

v0=symbols("v0")
v1=symbols("v1")
v2=symbols("v2")
v3=symbols("v3")

vx=[v0,v1,v2,v3]

for i in range(4):
    for j in range(4):
        for k in range(4):
            dx_res[i]-=ch[i,j,k]*vx[j]*vx[k]


In [15]:
for i in range(4):
    dx_res[0]=dx_res[0].simplify()
#dx_res=Matrix(dx_res)

In [22]:
from sympy import cse, ccode, fcode, pycode, MatrixSymbol
def c_code(expr):
    return ccode(cse(expr))

In [24]:
from sympy.codegen.ast import CodeBlock, Assignment
x0, x1, x2, x3 = symbols('x0 x1 x2 x3')
c = CodeBlock(Assignment(x0, dx_res[0]),Assignment(x1, dx_res[1]),Assignment(x2, dx_res[2]),Assignment(x3, dx_res[3]))
print(ccode(c))

x0 = rs*v0*v1/((rs - x)*x);
x1 = (1.0/2.0)*rs*pow(v0, 2)*(rs - x)/pow(x, 3) - pow(v1, 2)*(rs - x)*(-1.0/2.0*(rs - 2*x)*pow(x, 2)/pow(-rs*x + pow(x, 2), 2) - x/(-rs*x + pow(x, 2)))/x - pow(v2, 2)*(rs - x) - pow(v3, 2)*(rs - x)*pow(sin(y), 2);
x2 = -2*v1*v2/x + pow(v3, 2)*sin(y)*cos(y);
x3 = -2*v1*v3/x - 2*v2*v3*cos(y)/sin(y);


In [29]:
c2 = CodeBlock(Assignment(x0, dx_res[2]))
print(ccode(c2))

x0 = -2*v1*v2/x + pow(v3, 2)*sin(y)*cos(y);


In [26]:
for i in range(4):
    display(dx_res[i])

rs*v0*v1/((rs - x)*x)

rs*v0**2*(rs - x)/(2*x**3) - v1**2*(rs - x)*(-(rs - 2*x)*x**2/(2*(-rs*x + x**2)**2) - x/(-rs*x + x**2))/x - v2**2*(rs - x) - v3**2*(rs - x)*sin(y)**2

-2*v1*v2/x + v3**2*sin(y)*cos(y)

-2*v1*v3/x - 2*v2*v3*cos(y)/sin(y)

In [114]:
A = MatrixSymbol('A', 4, 1)
dx_resm=Matrix(dx_res)

from sympy.codegen.rewriting import optimize

optimize(dx_resm)

print(ccode(dx_resm,A))

TypeError: optimize() missing 1 required positional argument: 'optimizations'

In [77]:
ccode(dx_res)

'{rs*v0*v1/((rs - x)*x), (1.0/2.0)*rs*pow(v0, 2)*(rs - x)/pow(x, 3) - pow(v1, 2)*(rs - x)*(-1.0/2.0*(rs - 2*x)*pow(x, 2)/pow(-rs*x + pow(x, 2), 2) - x/(-rs*x + pow(x, 2)))/x - pow(v2, 2)*(rs - x) - pow(v3, 2)*(rs - x)*pow(sin(y), 2), -2*v1*v2/x + pow(v3, 2)*sin(y)*cos(y), -2*v1*v3/x - 2*v2*v3*cos(y)/sin(y)}'

In [17]:
#metric_to_Riemann_components(metric)